In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import shap
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# ---------------------------------------------------------------
# Load Training and Validation Data
# ---------------------------------------------------------------

train_df = pd.read_pickle("Datasets/train_df.pkl")
val_df = pd.read_pickle("Datasets/val_df.pkl")
test_df = pd.read_pickle("Datasets/test_df.pkl")

# Define target variable (log price)
y_train_log = np.log(train_df['Price Sold USD'])
y_val_log = np.log(val_df['Price Sold USD'])
y_test_log = np.log(test_df['Price Sold USD'])

In [ ]:
# ---------------------------------------------------------------
# XGBoost Modeling: One-Hot Encoding Categorical Features
# ---------------------------------------------------------------


# Define Categorical Columns for One-Hot Encoding
categorical_cols = [
    'Paint Imputed',
    'Material Imputed',
    'Country',
    'Auction House',
    'Alive Status'
]

# Ensure all categorical columns are string type (and no NaNs)
for df_ in [train_df, val_df]:
    df_[categorical_cols] = df_[categorical_cols].astype(str).fillna("Missing")


# One-hot encode categorical variables using training data
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(train_df[categorical_cols])

def encode_cats(df):
    encoded = encoder.transform(df[categorical_cols])
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)
    return pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

train_encoded = encode_cats(train_df)
val_encoded = encode_cats(val_df)
test_encoded = encode_cats(test_df)

# Final feature list
features = (
    list(encoder.get_feature_names_out(categorical_cols)) +
    ['Log Area', 'Artist Ordered Avg Price',
     'CPI_US', 'Artist Sale Count', 'Birth Period Ordinal',
     'Artist Cumulative Price']
)

# Targets (log-transformed)
X_train, y_train_log = train_encoded[features], np.log1p(train_encoded['Price Sold USD'])
X_val, y_val_log = val_encoded[features], np.log1p(val_encoded['Price Sold USD'])
X_test, y_test_log = test_encoded[features], np.log1p(test_encoded['Price Sold USD'])

In [ ]:
# ---------------------------------------------------------------
# XGBoost Modeling:  Define Objective Function for Optuna Tuning
# ---------------------------------------------------------------

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': 42,
        'verbosity': 0
    }

    model = XGBRegressor(**params)
    model.fit(
        X_train, y_train_log,
        eval_set=[(X_val, y_val_log)],
        verbose=False
    )

    val_preds_log = model.predict(X_val)
    return mean_absolute_error(y_val_log, val_preds_log)

# Run Optuna Optimization for XGBoost
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best MAE (log):", study.best_value)
print("Best params:", study.best_params)

In [ ]:
# ---------------------------------------------------------------
# XGBoost: Train Final Model with Best Parameters
# ---------------------------------------------------------------

best_xgb = XGBRegressor(**study.best_params, random_state=42)
best_xgb.fit(
    X_train, y_train_log,
    eval_set=[(X_val, y_val_log)],
    verbose=100
)

In [ ]:
# ---------------------------------------------------------------
# XGBoost: Evaluate the Optimized XGBoost Model on Validation Set
# ---------------------------------------------------------------

val_preds_log = best_xgb.predict(X_val)
val_preds = np.exp(val_preds_log)

mae_usd = mean_absolute_error(val_encoded['Price Sold USD'], val_preds)
mae_log = mean_absolute_error(y_val_log, val_preds_log)

print(f"Validation MAE (USD): ${mae_usd:,.2f}")
print(f"Validation MAE (Log): {mae_log:.4f}")

In [ ]:
# ---------------------------------------------------------------
# XGBoost: Evaluate on Test Set
# ---------------------------------------------------------------

test_preds_log = best_xgb.predict(X_test)
test_preds_usd = np.exp(test_preds_log)

mae_usd = mean_absolute_error(test_encoded['Price Sold USD'], test_preds_usd)
mae_log = mean_absolute_error(y_test_log, test_preds_log)

print(f"Validation MAE (USD): ${mae_usd:,.2f}")
print(f"Validation MAE (Log): {mae_log:.4f}")

In [ ]:
# ---------------------------------------------------------------
# XGBoost: Visualization – Residuals & Actual vs Predicted
# ---------------------------------------------------------------

# Residuals (Log scale)
residuals_log = y_val_log - val_preds_log

# Actual vs. Predicted (Log)
plt.figure(figsize=(8, 6))
plt.scatter(y_val_log, val_preds_log, alpha=0.3, edgecolor='k')
plt.plot([y_val_log.min(), y_val_log.max()], [y_val_log.min(), y_val_log.max()], 'r--', lw=2)
plt.xlabel('Actual Log Price')
plt.ylabel('Predicted Log Price')
plt.title('XGBoost: Actual vs Predicted Log Prices')
plt.grid(True)
plt.tight_layout()
plt.show()

# Residuals histogram
plt.figure(figsize=(8, 5))
plt.hist(residuals_log, bins=50, color='skyblue', edgecolor='black')
plt.title("XGBoost Residuals (Log Price)")
plt.xlabel("Actual - Predicted (Log Scale)")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------------
# XGBoost: SHAP Values
# ---------------------------------------------------------------

explainer = shap.Explainer(best_xgb, X_train, feature_names=features)
shap_values = explainer(X_val)

shap.summary_plot(shap_values, X_val, plot_type="bar")
shap.summary_plot(shap_values, X_val)